In [1]:
import pandas as pd
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.corpus import CorpusReader
import re
import numpy as np
import zipfile
import io

# Loading data

In [2]:
reader = CorpusReader(Loc.bundles_path/'n_nn/prepare/raw/raw_partial.zip')

In [3]:
frames = reader.get_frames()

In [4]:
df = frames.first()

In [5]:
df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id,original_word_id,original_sentence_id,original_paragraph_id,updated
0,0,0,0,0,1,По,ru,2,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,78,5,0,False
1,1,0,1,0,1,данным,ru,6,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,79,5,0,False
2,2,0,2,0,0,военных,ru,7,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,80,5,0,False
3,3,0,3,0,1,",",punct,1,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,81,5,0,False
4,4,0,4,0,1,арестованы,ru,10,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,82,5,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49970,49970,2095,15,672,1,заполнены,ru,9,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,2095,134095,6594,728,False
49971,49971,2095,16,672,1,дорогостоящим,ru,13,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,2095,134096,6594,728,False
49972,49972,2095,17,672,1,высокотехнологичным,ru,19,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,True,0,2095,134097,6594,728,False
49973,49973,2095,18,672,0,оборудованием,ru,13,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,2095,134098,6594,728,False


In [42]:
 with open(Loc.bundles_path/'n_nn/partial/src.parquet', 'rb') as f:
     buffer = io.BytesIO(f.read())
     src_df = pd.read_parquet(buffer)

In [43]:
src_df

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id,original_word_id,original_sentence_id,original_paragraph_id,updated
0,0,0,0,0,1,По,ru,2,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,78,5,0,False
1,1,0,1,0,1,данным,ru,6,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,79,5,0,False
2,2,0,2,0,0,военных,ru,7,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,80,5,0,False
3,3,0,3,0,1,",",punct,1,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,81,5,0,False
4,4,0,4,0,1,арестованы,ru,10,c1df6b29-a5b3-4e3f-b125-a4b481bb568d,raw_partial.zip,lenta.base.zip,False,0,0,82,5,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249913,249913,222471,3,221508,1,подробности,ru,11,c1efe2b2-6450-4048-9ac0-ba27b838672f,raw_partial.zip,lenta.base.zip,False,1,4166,275191,199043,195310,False
249914,249914,222471,4,221508,1,уголовного,ru,10,c1efe2b2-6450-4048-9ac0-ba27b838672f,raw_partial.zip,lenta.base.zip,False,1,4166,275192,199043,195310,False
249915,249915,222471,5,221508,1,дела,ru,4,c1efe2b2-6450-4048-9ac0-ba27b838672f,raw_partial.zip,lenta.base.zip,True,1,4166,275193,199043,195310,False
249916,249916,222471,6,221508,0,Фатеева,ru,7,c1efe2b2-6450-4048-9ac0-ba27b838672f,raw_partial.zip,lenta.base.zip,False,1,4166,275194,199043,195310,False


In [44]:
 with open(Loc.bundles_path/'n_nn/partial/index.parquet', 'rb') as f:
     buffer = io.BytesIO(f.read())
     index_df = pd.read_parquet(buffer)

In [45]:
index_df

,word_id,sentence_id,label,reference_sentence_id,split
sample_id,,,,,
0,9,0,0,0,train
1,68,1,0,1,train
2,76,2,0,2,train
3,103,3,0,3,test
4,111,4,0,4,display
...,...,...,...,...,...
13635,249875,222469,1,4164,test
13636,249881,222469,1,4164,display
13637,249882,222469,1,4164,train


In [50]:
def get_ratio_of_labels(df):
    ratio = (df['label'] == 1).sum() / (df['label'] == 0).sum()
    if ratio > 1:
        return 1/ratio
    return ratio

In [51]:
get_ratio_of_labels(index_df)

0.6419886842422053

In [61]:
index_df[index_df['split'] == 'test'].iloc[:2000]

,word_id,sentence_id,label,reference_sentence_id,split
sample_id,,,,,
3,103,3,0,3,test
7,171,7,0,7,test
8,172,7,0,7,test
11,214,10,0,10,test
12,238,11,0,11,test
...,...,...,...,...,...
6732,124936,72018,1,2292,test
6739,125070,72022,1,2296,test
6740,125073,72022,1,2296,test


In [40]:
(pd.merge(src_df, index_df[index_df.split == 'test'], how='inner', on='word_id')['label_y'] == 1).sum()

1576

# Extracting words

In [14]:
from importlib import reload
from tg.grammar_ru.ml.tasks.n_nn import word_normalizer
from tg.grammar_ru.ml.tasks.n_nn import bundle
from research.common import build_bundle
from tg.grammar_ru.ml.tasks.n_nn.regular_expressions import single_n_regex, double_n_regex

In [15]:
LoggingRegexNormalizer = reload(word_normalizer).LoggingRegexNormalizer
build_dictionary = reload(bundle).build_dictionary

In [8]:
def _extract_words(dfs):
    res = []
    for df in dfs:
        a = df[df['word'].str.lower().str.contains('скважинн')]
        if a.shape[0]:
            res.append(df)
            print(a)

    return res

x = _extract_words(build_bundle.read_data(build_bundle.LENTA_CORPUS_PATH))

  0%|          | 0/869 [00:00<?, ?it/s]

         word_id  sentence_id  word_index  paragraph_id  word_tail  \
6458644  6458644      6372876          14       6368524          1   

               word word_type  word_length  original_word_id  \
6458644  скважинной        ru           10             90554   

         original_sentence_id  original_paragraph_id  updated  \
6458644                  4786                    434    False   

                                      file_id       corpus_id  
6458644  d69ad8ba-f264-43a9-84dd-83663ce10719  lenta.base.zip  
           word_id  sentence_id  word_index  paragraph_id  word_tail  \
25076319  25076319     24983354           5      24978651          1   

                word word_type  word_length  original_word_id  \
25076319  скважинной        ru           10             98083   

          original_sentence_id  original_paragraph_id  updated  \
25076319                  5118                    415    False   

                                       file_id       corpus_id

In [11]:
a = x[1]
a[a['word'].str.contains('скважин')]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
25011792,25011792,24980054,18,24978386,1,скважины,ru,8,33556,1818,150,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25076260,25076260,24983351,8,24978651,1,скважин,ru,7,98024,5115,415,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25076319,25076319,24983354,5,24978651,1,скважинной,ru,10,98083,5118,415,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25080141,25080141,24983556,13,24978666,1,скважин,ru,7,101905,5320,430,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25091634,25091634,24984159,10,24978713,1,скважины,ru,8,113398,5923,477,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25091660,25091660,24984160,16,24978713,1,скважин,ru,7,113424,5924,477,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25091784,25091784,24984166,14,24978713,1,скважины,ru,8,113548,5930,477,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25103405,25103405,24984737,1,24978761,1,скважина,ru,8,125169,6501,525,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip
25103416,25103416,24984738,2,24978761,1,скважины,ru,8,125180,6502,525,False,8811eaa5-41ca-41e9-8b45-295b066927af,lenta.base.zip


In [16]:
normalizer = LoggingRegexNormalizer()
words = list(filter(lambda x: len(x) > 3, build_dictionary(
    build_bundle.read_data(build_bundle.LENTA_CORPUS_PATH),
    normalizer))
    )

  0%|          | 0/869 [00:00<?, ?it/s]

In [20]:
normalizer._map

defaultdict(list,
            {'шестиствольные': ['шестистволь'],
             'внепартийные': ['внепартий'],
             'воздушно-топливной': ['воздушно-топлив'],
             'соединительном': ['соединитель'],
             'ремонтопригоднее': ['ремонтопригод'],
             'водочными': ['водоч'],
             'межтерриториальных': ['межтерриториаль'],
             'глазных': ['глаз'],
             '72-этажные': ['72-этаж'],
             'инвалидную': ['инвалид'],
             'постатейном': ['постатей'],
             'векторный': ['вектор'],
             'арушение': ['аруше'],
             'противогололедным': ['противогололед'],
             'непосильные': ['непосиль'],
             'уездном': ['уезд'],
             'газосборными': ['газосбор'],
             'корректного': ['коррект'],
             'слоганом': ['слога'],
             'Музыкального': ['музыкаль'],
             'контекстуальные': ['контекстуаль'],
             'комфортабельнее': ['комфортабель'],
             'инди

In [17]:
from collections import defaultdict
def reverse_dictionary(dct):
    reversed = defaultdict(list)
    for key, values in dct.items():
        for value in values:
            reversed[value.lower()].append(key)

    return reversed

In [18]:
reversed_map = reverse_dictionary(normalizer._map)

In [21]:
reversed_map

defaultdict(list,
            {'шестистволь': ['шестиствольные',
              'шестиствольная',
              'шестиствольный',
              'шестиствольную',
              'шестиствольных',
              'шестиствольными',
              'шестиствольной',
              'шестиствольным'],
             'внепартий': ['внепартийные',
              'внепартийного',
              'внепартийным',
              'внепартийную',
              'внепартийной',
              'внепартийных',
              'внепартийными',
              'внепартийное',
              'внепартийная',
              'внепартийный'],
             'воздушно-топлив': ['воздушно-топливной'],
             'соединитель': ['соединительном',
              'Соединительные',
              'соединительной',
              'соединительным',
              'соединительный',
              'соединительная',
              'соединительные',
              'соединительного',
              'соединительное',
              'соединительную',
 

In [33]:
words_set = set(words)

In [34]:
words_set

{'переизбра',
 'обводне',
 'заиле',
 'амнистирова',
 'высокотехнологич',
 'довнесе',
 'мумифицирова',
 '10-миллио',
 'неподтвержде',
 'перекреще',
 'укорене',
 'неуказа',
 'растле',
 'кипяче',
 'антикоррупцио',
 'кувши',
 'склоне',
 'отреше',
 'шифрова',
 'райо',
 'мезо',
 'деградирова',
 'антиге',
 'озаглавле',
 'генерирова',
 'комбинирова',
 'порожде',
 'чемода',
 'сближе',
 'аккумулирова',
 'отклоне',
 'основа',
 'полы',
 'оскорбле',
 'предопределе',
 'двухсторо',
 'операцио',
 'времене',
 'произведе',
 'павильо',
 'курга',
 '21-миллио',
 'разграбле',
 'смартфо',
 'переучрежде',
 'рассредоточе',
 'притупле',
 'призна',
 'обрудова',
 'креме',
 'нукло',
 'доли',
 'моторизирова',
 'униже',
 'выго',
 'неразруше',
 'полива',
 'канализацио',
 'спрямле',
 'структурирова',
 'засекрече',
 'замыле',
 'зафиксирова',
 'соверше',
 'поруче',
 'бура',
 'матерщи',
 'сосредоточе',
 'лурма',
 'эскадро',
 'недовыполне',
 'жарго',
 'неосвое',
 'принесе',
 'лише',
 'внесе',
 'смета',
 'прославле',
 'сои

In [44]:
reversed_map['маловероят']  # маловероятнными??

['маловероятное',
 'маловероятном',
 'маловероятную',
 'маловероятной',
 'маловероятного',
 'маловероятные',
 'Маловероятная',
 'маловероятная',
 'Маловероятным',
 'Маловероятными',
 'маловероятный',
 'маловероятным',
 'маловероятных',
 'Маловероятной',
 'маловероятными',
 'маловероятнными']

In [45]:
reversed_map['10-минут']  # 10-минутнную??

['10-минутных',
 '10-минутный',
 '10-минутном',
 '10-минутного',
 '10-минутное',
 '10-минутному',
 '10-минутная',
 '10-минутные',
 '10-минутной',
 '10-минутным',
 '10-минутными',
 '10-минутную',
 '10-минутнную']

In [46]:
reversed_map['подда']
# кажется, должно везде писаться две н
# корень -подданн- + суффикс -н- 

['подданым',
 'подданого',
 'Подданые',
 'подданый',
 'подданые',
 'поддаными',
 'подданая',
 'подданых',
 'Подданные',
 'подданном',
 'подданную',
 'подданным',
 'Подданный',
 'подданные',
 'подданной',
 'Подданная',
 'Подданных',
 'подданных',
 'Подданного',
 'подданная',
 'Подданному',
 'подданному',
 'подданный',
 'подданными',
 'подданного']

In [47]:
reversed_map['15-лет']  # 15-летнний??

['15-летнему',
 '15-летний',
 '15-летних',
 '15-летнего',
 '15-летним',
 '15-летней',
 '15-летную',
 '15-летнем',
 '15-летние',
 '15-летнее',
 '15-летними',
 '15-летнний']

In [48]:
reversed_map['высокотехнологич']  # высокотехнологичнного??

['высокотехнологичных',
 'высокотехнологичной',
 'высокотехнологичную',
 'высокотехнологичный',
 'Высокотехнологичной',
 'высокотехнологичным',
 'высокотехнологичная',
 'высокотехнологичного',
 'высокотехнологичном',
 'высокотехнологичные',
 'Высокотехнологичные',
 'высокотехнологичному',
 'Высокотехнологичная',
 'высокотехнологичными',
 'Высокотехнологичную',
 'Высокотехнологичное',
 'высокотехнологичное',
 'Высокотехнологичный',
 'высокотехнологичнного']

In [51]:
reversed_map['антикоррупцио']  # отыменное прилагательное, суффикс должен быть суффикс -онн-, коррупционный

['Антикоррупционых',
 'антикоррупционого',
 'антикоррупционые',
 'антикоррупциоными',
 'антикоррупционному',
 'антикоррупционном',
 'антикоррупционным',
 'антикоррупционную',
 'антикоррупционный',
 'антикоррупционная',
 'Антикоррупционная',
 'антикоррупционное',
 'антикоррупционными',
 'Антикоррупционные',
 'Антикоррупционной',
 'Антикоррупционное',
 'антикоррупционного',
 'антикоррупционных',
 'Антикоррупционный',
 'антикоррупционной',
 'антикоррупционные',
 'Антикоррупционную',
 'Антикоррупционного',
 'Антикоррупционным',
 'Антикоррупционном']

In [54]:
reversed_map['антикорупцио']  # :)

['антикорупционного', 'антикорупционная']

In [55]:
reversed_map['озаглавле']  # кажется, должно писаться две н, т.к. есть приставка о-

['озаглавленого',
 'озаглавленом',
 'озаглавленым',
 'озаглавленная',
 'озаглавленному',
 'озаглавленный',
 'озаглавленные',
 'озаглавленным',
 'озаглавленном',
 'озаглавленную',
 'озаглавленных',
 'озаглавленной',
 'озаглавленное',
 'озаглавленного']

In [57]:
reversed_map['полы']  # стоит ли обрабатывать отыменные прилагательные с корнем, оканчивающимся на -н-?

['полыней', 'полынной', 'Полынный', 'полынными', 'полынных']

In [58]:
reversed_map['двухсторо']  # второй корень -сторон-, кажется, должно быть две н

['двухсторонее',
 'двухстороннему',
 'Двухсторонние',
 'двухсторонним',
 'двухстороннее',
 'двухсторонними',
 'Двухстороннее',
 'двухстороннего',
 'двухсторонних',
 'двухсторонние',
 'двухсторонний',
 'двухсторонней',
 'двухстороннем']

In [59]:
reversed_map['операцио']  # отыменное прилагательное, суффикс -онн-

['операционого',
 'операционным',
 'операционной',
 'операционные',
 'Операционное',
 'операционное',
 'Операционных',
 'операционными',
 'Операционный',
 'операционного',
 'операционном',
 'Операционные',
 'Операционную',
 'операционная',
 'Операционному',
 'операционную',
 'Операционным',
 'Операционной',
 'операционному',
 'Операционная',
 'операционных',
 'операционный']

In [66]:
reversed_map['21-миллио']  # -милион- корень + суффикс -н-

['21-миллионого', '21-миллионного']

In [68]:
reversed_map['10-миллио']  # то же самое

['10-миллионая',
 '10-миллионной',
 '10-миллионным',
 '10-миллионный',
 '10-миллионное',
 '10-миллионную',
 '10-миллионные',
 '10-миллионных',
 '10-миллионная',
 '10-миллионного']

In [ ]:
reversed_map['10-милио']  # :)

['10-милионной']

In [71]:
reversed_map['креме']  # тут речь о населенном пункте, или я не понимаю смысл слова "Кременом"

['Кременом', 'Кременной', 'Кременная']

Просмотрел буквально чуть меньше ста слов первых, нашел столько ошибок. Конечно, если еще немалая доля ложно-положительных срабатываний, буду думать, как исправлять.